# 기본 전처리 & 단어 사전 생성 코드
### 코더 : 박연우
### 버전 01, 02

In [36]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np

In [45]:
train_data_path ="~/aiffel/dktc/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [46]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

In [47]:
corpus.head()

0    지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
Name: conversation, dtype: object

In [48]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=1000, 
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  1 210   1 ...   1  41   1]
 [  1   1   1 ...   1   1 396]
 [  1  96   1 ...   1 125   1]
 ...
 [485   3 113 ...  27   1   1]
 [  1   1 152 ... 350   1   1]
 [  1   1   1 ... 156 114   1]] <keras_preprocessing.text.Tokenizer object at 0x7d0ec10220d0>


In [49]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 내가
3 : 너
4 : 좀
5 : 다
6 : 왜
7 : 나
8 : 진짜
9 : 야
10 : 지금


In [50]:
X_train = tensor[:3500]
y_train = train_data['class'][:3500]
X_val = tensor[3500:3850]
y_val = train_data['class'][3500:3850]
X_test = tensor[3850:]
y_test = train_data['class'][3850:]

In [51]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

## 버전 01

### 전처리

In [22]:
# 전처리 함수 수정 ver.

def preprocess_sentence(sentence):
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)
    
    sentence = sentence.strip()
    return sentence


In [ ]:

def load_conversations_from_df(corpus_df):
    conv_data = []

    for index, row in corpus_df.iterrows():  # 수정된 부분
        conversations = row['conversation']  # 수정된 부분
        conv_data.append(preprocess_sentence(conversations))  # 전처리 함수 적용
    return conv_data

In [25]:

conv_data = load_conversations_from_df(train_data)

print('전체 샘플 수:', len(conv_data))

전체 샘플 수: 3950


In [27]:
# 한국어 전처리 함수 적용 결과 

print('전처리 후의 1081번째 샘플: {}'.format(conv_data[1081]))
print('전처리 후의 500번째 샘플: {}'.format(conv_data[500]))


전처리 후의 1081번째 샘플: 아까 시에 배민에서 스테이크 덮밥 시켰던 사람인데요 시요 ? 벌써 시간이나 지났는데 . 무슨 일이신데요 ? 손님한테 그렇게 밖에 말을 못해요 ? 아닙니다 죄송합니다 오늘 스테이크에 제대로 된 고기 쓰시는 것 맞아요 ? 네 저희는 제대로 수입된 고기 납품받아 조리하고 있습니다 . 고기가 너무 질겨서 아이가 씹지도 못했어요 . 이거 환불해주세요 . 손님 배달 특성상 조리 후에 바로 배달된다해도 고기의 육질상태가 시간이 흐름에 따라 저하되는 것은 당연합니다 . 지금 말 다했어요 ? 그리고 반 이상 드셔놓고 환불해달라고 하시면 어떡해요 . 그럼 지금 고객인 내가 잘못했다는 거에요 ? 이거 안되겠네 . 당장 업체 본사에 전화하고 배민 고객센터에도 다 신고할거에요 . 당신 이 바닥에서 장사 못 해 이제 .
전처리 후의 500번째 샘플: 자기야 나 필요한거 있는뎅 . ! 어떤거 ? 음 나 가방 필요해서 그런데 만원만 주랑 웅 . ? 갑자기 ? 자기 며칠 전에도 가방 사줬잖아 그거는 내 생일선물이였구우 이번꺼는 그냥 여자친구한테 선물 줄 수 있는거잖아앙 ! 음자기가 이런식으로 매년 나한테 요구하는 돈이 얼만지 알아 ? 년마다 단위인데 우리 미래 생각하려면 돈 아껴야지 . ! 헐 모야잉 자기야 어차피 결혼하면 자기 돈 내 돈 상관 없이 같이 쓸텐데 나한테 쓰는게 아까워 ? ! 가방 살 돈 안주면 화낸다 ? ! 그게 아니라 그럼 사죠 돈 보내놔 알겠징 ? ! 자기야 사랑행 하 그래


In [29]:
# SubwordTextEncoder

import tensorflow_datasets as tfds
print("데이터셋에 대해서 Vocabulary 생성 중...")

# 단일 데이터셋에 대해서 Vocabulary 생성
# tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(conv_data, target_vocab_size=2**13)

print("Vocabulary 생성 완료 !")

데이터셋에 대해서 Vocabulary 생성 중...
Vocabulary 생성 완료 !


In [30]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]


In [31]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [7999]
END_TOKEN의 번호 : [8000]


In [32]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8001


In [34]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환

# print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
# print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))


# 병렬 데이터 -> 단일 시퀀스 데이터 변경 후
print('정수 인코딩 후의 21번째 conv_data 샘플: {}'.format(tokenizer.encode(conv_data[21])))


정수 인코딩 후의 21번째 conv_data 샘플: [4, 434, 433, 3893, 124, 32, 5166, 1603, 165, 1, 4075, 26, 14, 21, 1545, 1, 5, 11, 2819, 3519, 197, 765, 113, 2228, 1, 3, 14, 2924, 3786, 2, 11, 2819, 6620, 2593, 2, 58, 3385, 880, 32, 189, 2061, 134, 1938, 1800, 4508, 2, 684, 25, 3786, 2, 421, 208, 135, 1, 5, 21, 265, 136, 1, 704, 2819, 3745, 3137, 2, 488, 762, 13]


In [37]:

conv_data_lengths = [len(data) for data in conv_data]

# 통계값 계산
min_conv_data_length = np.min(conv_data_lengths)
max_conv_data_length = np.max(conv_data_lengths)
avg_conv_data_length = np.mean(conv_data_lengths)

print(f"데이터 길이 - 최소: {min_conv_data_length}, 최대: {max_conv_data_length}, 평균: {avg_conv_data_length}")


데이터 길이 - 최소: 41, 최대: 909, 평균: 235.78683544303797


In [38]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
# == 최대 시퀀스 길이

MAX_LENGTH = 400

print(MAX_LENGTH)

400


In [39]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
## 단일 시퀀스 데이터이므로 그에 맞게 변경 

def tokenize_and_filter(chat_data):
  tokenized_chat_data = []
  
  for sentence in chat_data:
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN

    # 최대 길이 78 이하인 경우에만 데이터셋으로 허용
    if len(sentence) <= MAX_LENGTH :
      tokenized_chat_data.append(sentence)

  
  # 최대 길이로 모든 데이터셋을 패딩
  tokenized_chat_data = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_chat_data, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_chat_data

In [40]:
tokenized_and_filtered_conv_data = tokenize_and_filter(conv_data)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 샘플 개수: {}'.format(len(tokenized_and_filtered_conv_data)))

단어장의 크기 : 8001
필터링 후의 샘플 개수: 3950


## 버전 02 

### 3429 번째 샘플 예시

In [41]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter

# 1. 데이터 샘플을 DataFrame으로 변환
data = [
    [3429, "협박 대화", "너 우리 돈 사기치고 두눈뜨고 편하게살줄알았냐? 돌려주려했는데 사정이안좋아서 한달만더기다려주라 이새끼 아직 정신못차렸네 누가 그말을 믿어줄까? 니 와이프 중국에 넘기면그만이야 눈 하나에 신장하나만 팔면 딱 갚겠네 제발 그러지마 기회를줘 그럼 먼저똑바로살았어야지 아니면 너가 중국에 가던지 지금 우리말장난같애? 이 칼보여? 무릎꿇고빌게 죄송합니다 저걸로 니 배때지 쑤셔버릴껀데 일단 저새끼 입막자"]
]

train_df = pd.DataFrame(data, columns=['id', 'class', 'conversation'])

# 2. 데이터 정제 함수
def clean_text(text):
    return text.replace(" ", "")  # 예시로 공백 제거

# 3. 단어 사전 구축
def build_vocab(data):
    okt = Okt()
    all_words = []
    
    for sentence in data:
        cleaned = clean_text(sentence)  # 데이터 정제
        tokens = okt.morphs(cleaned)  # 형태소 토큰화
        all_words.extend(tokens)  # 모든 단어를 리스트에 추가
    
    # 단어의 빈도수 계산
    word_counts = Counter(all_words)
    
    # 고유한 단어 리스트 만들기 (빈도수 기준으로 정렬)
    vocab = {word: i for i, (word, _) in enumerate(word_counts.most_common())}
    
    return vocab

# 4. 단어 사전 생성
vocab = build_vocab(train_df['conversation'].tolist())

# 5. 단어 사전 출력
print("단어 사전:", vocab)
print("단어 수:", len(vocab))

# 6. 인덱스 변환 예시 (문장을 숫자 시퀀스로 변환)
def sentence_to_sequence(sentence, vocab):
    tokens = Okt().morphs(clean_text(sentence))  # 문장 토큰화
    return [vocab.get(token, -1) for token in tokens]  # 단어를 인덱스로 변환, 미등록 단어는 -1로 설정

# 예시 문장 변환
example_sentence = train_df['conversation'][0]  # 첫 번째 대화 내용
sequence = sentence_to_sequence(example_sentence, vocab)
print("숫자 시퀀스:", sequence)


단어 사전: {'?': 0, '너': 1, '만': 2, '새끼': 3, '네': 4, '그': 5, '니': 6, '중국': 7, '에': 8, '우리': 9, '돈': 10, '사기치고두': 11, '눈뜨고': 12, '편하게': 13, '살줄': 14, '알았냐': 15, '돌려주려': 16, '했는데': 17, '사': 18, '정이': 19, '안좋아서': 20, '한': 21, '달': 22, '더기': 23, '다려주': 24, '라이': 25, '아직': 26, '정신': 27, '못': 28, '차렸': 29, '누가': 30, '말': 31, '을': 32, '믿어줄까': 33, '와이프': 34, '넘기면': 35, '만이': 36, '야': 37, '눈': 38, '하나': 39, '신': 40, '장하나': 41, '팔면': 42, '딱': 43, '갚겠': 44, '제발그러지마': 45, '기회': 46, '를': 47, '줘': 48, '그럼': 49, '먼저': 50, '똑바로': 51, '살았어야지': 52, '아니면': 53, '가': 54, '에가': 55, '던지': 56, '지금': 57, '우리말': 58, '장': 59, '난': 60, '같': 61, '애': 62, '이': 63, '칼': 64, '보여': 65, '무릎': 66, '꿇고': 67, '빌게': 68, '죄송합니다': 69, '저걸': 70, '로': 71, '배때': 72, '지': 73, '쑤셔': 74, '버릴': 75, '껀': 76, '데일': 77, '단': 78, '저': 79, '입': 80, '막자': 81}
단어 수: 82
숫자 시퀀스: [1, 9, 10, 11, 12, 13, 14, 15, 0, 16, 17, 18, 19, 20, 21, 22, 2, 23, 24, 25, 3, 26, 27, 28, 29, 4, 30, 5, 31, 32, 33, 0, 6, 34, 7, 8, 35, 5, 36, 37, 38, 39, 8, 40, 41, 

In [53]:

data = corpus 

train_df = pd.DataFrame(data, columns=['id', 'class', 'conversation'])

# 2. 데이터 정제 함수
def clean_text(sentence):
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
#     return text.replace(" ", "")  # 예시로 공백 제거
    return sentence

# 3. 단어 사전 구축
def build_vocab(data):
    okt = Okt()
    all_words = []
    
    for sentence in data:
        cleaned = clean_text(sentence)  # 데이터 정제
        tokens = okt.morphs(cleaned)  # 형태소 토큰화
        all_words.extend(tokens)  # 모든 단어를 리스트에 추가
    
    # 단어의 빈도수 계산
    word_counts = Counter(all_words)
    
    # 고유한 단어 리스트 만들기 (빈도수 기준으로 정렬)
    vocab = {word: i for i, (word, _) in enumerate(word_counts.most_common())}
    
    return vocab

# 4. 단어 사전 생성
print("단어 사전 생성 중 ...")
vocab = build_vocab(train_df['conversation'].tolist())

# 5. 단어 사전 출력
print("단어 사전:", vocab)
print("단어 수:", len(vocab))

# 6. 인덱스 변환 예시 (문장을 숫자 시퀀스로 변환)
def sentence_to_sequence(sentence, vocab):
    tokens = Okt().morphs(clean_text(sentence))  # 문장 토큰화
    return [vocab.get(token, -1) for token in tokens]  # 단어를 인덱스로 변환, 미등록 단어는 -1로 설정


단어 사전 생성 중 ...
단어 사전: {'.': 0, '?': 1, '가': 2, '이': 3, '너': 4, '!': 5, '내': 6, '에': 7, '야': 8, '네': 9, '아': 10, '나': 11, '도': 12, '안': 13, '말': 14, '왜': 15, '을': 16, '거': 17, '는': 18, '좀': 19, '못': 20, '돈': 21, '다': 22, '뭐': 23, '니': 24, '은': 25, '요': 26, '들': 27, '저': 28, '만': 29, '진짜': 30, '해': 31, '죄송합니다': 32, '제': 33, '지금': 34, '아니': 35, '잘': 36, '를': 37, '고': 38, '한': 39, '한테': 40, '그래': 41, '님': 42, '그': 43, '로': 44, '우리': 45, '그럼': 46, '무슨': 47, '게': 48, '이야': 49, '것': 50, '일': 51, '그냥': 52, '으로': 53, '봐': 54, '사람': 55, '제발': 56, '알': 57, '할': 58, '하고': 59, '까지': 60, '오늘': 61, '에서': 62, '더': 63, '지': 64, '그렇게': 65, '여기': 66, '랑': 67, '돼': 68, '새끼': 69, '생각': 70, '어떻게': 71, '씨': 72, '어': 73, '집': 74, '너무': 75, '때': 76, '하면': 77, '아니야': 78, '와': 79, '정말': 80, '없어': 81, '수': 82, '애': 83, '빨리': 84, '하는': 85, '면': 86, '이번': 87, '줄': 88, '하': 89, '하나': 90, '그런': 91, '이렇게': 92, '회사': 93, '시간': 94, '그게': 95, '그건': 96, '있어': 97, '친구': 98, '서': 99, '넌': 100, '대리': 101, '인데': 102, '응': 103

In [54]:
# 예시 문장 변환
example_sentence = train_df['conversation'][40]  # 첫 번째 대화 내용
sequence = sentence_to_sequence(example_sentence, vocab)
print("숫자 시퀀스:", sequence)

숫자 시퀀스: [141, 965, 285, 108, 149, 1, 643, 365, 342, 201, 1554, 54, 12487, 183, 13, 365, 342, 3272, 4543, 24, 2, 12488, 916, 74, 7, 214, 12489, 810, 3, 48, 30, 3273, 6, 2725, 304, 10, 3, 519, 294, 100, 939, 1841, 215, 282, 12490, 111, 197, 5325, 282, 8425, 29, 229, 38, 917]
